# Demand Forecast Model Sensitivity Analysis: NE and SE Electrical Regions as Case Studies 

In [4]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
from sklearn import linear_model

## North East Electrical Region

In [66]:
dfn = pd.read_excel('NESE Demand.xlsx', 'NE_Daily', parse_dates = ['Local date'])
dfn.head()

,Region,Local date,Time zone,Demand Forecast,Demand,GHI,Wind Speed,Relative Humidity,Temperature,Pressure
0,NE,2015-07-01,Eastern,367230,375496,170.583333,0.731510,97.353229,18.182292,989.296875
1,NE,2015-07-02,Eastern,362810,364451,298.010417,0.473177,83.525286,17.424479,990.911458
2,NE,2015-07-03,Eastern,316550,330178,349.164062,0.542708,75.774635,17.520833,995.520833
3,NE,2015-07-04,Eastern,311490,301690,170.221354,0.519792,84.102917,17.481771,996.250000
4,NE,2015-07-05,Eastern,321810,323679,296.351562,0.576823,83.924714,19.359375,997.343750


In [68]:
def data_cleaner(data_frame):
    for column in data_frame:
        if (column == 'Demand'):
            continue
        elif (column == 'Local date'):
            continue
        else:
            max_thresold = data_frame[column].quantile(0.95)
            min_thresold = data_frame[column].quantile(0.05)
            data_frame = data_frame[(data_frame[column]<max_thresold) & (data_frame[column]>min_thresold)]
    return data_frame

In [86]:
dfnee = dfn.drop(columns = ['Region','Time zone'], axis = 1)
print(dfnee.head())
dfnee.describe()

  Local date  Demand Forecast  Demand         GHI  Wind Speed  \
0 2015-07-01           367230  375496  170.583333    0.731510   
1 2015-07-02           362810  364451  298.010417    0.473177   
2 2015-07-03           316550  330178  349.164062    0.542708   
3 2015-07-04           311490  301690  170.221354    0.519792   
4 2015-07-05           321810  323679  296.351562    0.576823   

   Relative Humidity  Temperature    Pressure  
0          97.353229    18.182292  989.296875  
1          83.525286    17.424479  990.911458  
2          75.774635    17.520833  995.520833  
3          84.102917    17.481771  996.250000  
4          83.924714    19.359375  997.343750  


,Demand Forecast,Demand,GHI,Wind Speed,Relative Humidity,Temperature,Pressure
count,2009.000000,2009.000000,2009.000000,2009.000000,2009.000000,2009.000000,2009.000000
mean,328433.648581,332912.298656,160.903136,0.976342,82.337145,8.804022,998.877320
std,46984.192070,47541.844372,88.952952,0.420929,8.016374,9.878617,8.020579
min,219380.000000,227884.000000,8.593750,0.371354,55.211302,-18.268229,970.520833
25%,295300.000000,299187.000000,84.221354,0.664844,76.601536,0.687760,993.723958
50%,321510.000000,325672.000000,149.848958,0.872656,82.426510,8.937500,998.671875
75%,354330.000000,358723.000000,232.419271,1.187240,88.270026,18.014583,1004.291667
max,511920.000000,517594.000000,362.734375,3.277604,99.684036,26.368750,1023.424479


In [69]:
#dfne = data_cleaner(dfnee)
dfne = dfnee
dfne.shape

(2009, 8)

In [70]:
dfne.isnull().any()

Local date           False
Demand Forecast      False
Demand               False
GHI                  False
Wind Speed           False
Relative Humidity    False
Temperature          False
Pressure             False
dtype: bool

In [ ]:
sns.pairplot(dfne, kind = 'scatter', diag_kind = 'kde')

In [72]:
def season_extractor(dataframe, start_month:int, start_day:int, end_month:int, end_day:int):
    dff = dataframe[dataframe['Local date'].apply(lambda x: x.month>=start_month and x.month<=end_month)]
    dff = dff[dff['Local date'].apply(lambda x: x.day>=start_day if x.month==start_month else True)]
    dff = dff[dff['Local date'].apply(lambda x: x.day<=end_day if x.month==end_month else True)]
    return dff

### Model Selection

In [77]:
def train_model(variable, target, model):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(variable, target, test_size = 0.3, random_state=0)
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [87]:
X = dfne.drop(['Demand', 'Demand Forecast', 'Local date'], axis = 1)
y = dfne['Demand']
X.shape, y.shape

((2009, 5), (2009,))

In [108]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
train_model(X,y,model_lr)

0.0625481722746336

In [109]:
from sklearn.linear_model import Ridge
ridge_reg= Ridge(alpha=50, max_iter=100, tol=0.1)
train_model(X,y,ridge_reg)

0.0622511071327857

In [110]:
from sklearn.linear_model import Lasso
lasso_reg= Lasso(alpha=50, max_iter=100, tol=0.1)
train_model(X,y,lasso_reg)

0.06228146637407317

In [111]:
from sklearn.model_selection import cross_val_score
def get_scores(variables, target):
    scores = cross_val_score(ridge_reg, variables, target, cv = 5)
    return np.max(scores)

### NE - Summer Calculations

In [112]:
df_summer = season_extractor(dfne, 6, 21, 9, 22)
print(df_summer.shape)
summer_variables = df_summer.drop(['Local date','Demand Forecast','Demand'], axis = 'columns')
summer_target = df_summer['Demand']
summer_variables.head()

(554, 8)


,GHI,Wind Speed,Relative Humidity,Temperature,Pressure
0,170.583333,0.731510,97.353229,18.182292,989.296875
1,298.010417,0.473177,83.525286,17.424479,990.911458
2,349.164062,0.542708,75.774635,17.520833,995.520833
3,170.221354,0.519792,84.102917,17.481771,996.250000
4,296.351562,0.576823,83.924714,19.359375,997.343750


In [ ]:
%matplotlib inline
sns.pairplot(df_summer.drop('Local date', axis = 1))

In [114]:
summer_scores = get_scores(summer_variables, summer_target)
summer_scores

0.8143397227218738

In [115]:
def wind_temp_calculator(variables, target):
    variables_wt = variables.drop(columns = ['GHI','Relative Humidity','Pressure'])
    scores_wt = get_scores(variables_wt, target)
    return scores_wt    

In [116]:
wind_temp_calculator(summer_variables, summer_target)

0.8178716836191996

In [117]:
def temp_calculator(variables, target):
    variables_t = variables.drop(columns = ['GHI','Relative Humidity','Pressure','Wind Speed'])
    scores_t = get_scores(variables_t, target)
    return scores_t

In [118]:
temp_calculator(summer_variables, summer_target)

0.8195658497630626

In [119]:
def no_temp_calculator(variables, target): 
    variables_nt = variables.drop(columns=['Temperature'])
    scores_nt = get_scores(variables_nt, target)
    return scores_nt

In [120]:
no_temp_calculator(summer_variables, summer_target)

0.4049745331573664

### NE - Winter Calculations

In [ ]:
dfff = dfne[dfne['Local date'].apply(lambda x: x.month==1 or x.month==2 or x.month==3 or x.month==12)]
dfff = dfff[dfff['Local date'].apply(lambda x: x.day<=21 if x.month==3 else True)]
dfff = dfff[dfff['Local date'].apply(lambda x: x.day>=21 if x.month==12 else True)]
df_winter = dfff
winter_variables = df_winter.drop(['Local date','Demand Forecast','Demand'], axis = 1)
winter_target = df_winter['Demand']
%matplotlib inline
sns.pairplot(df_winter.drop('Local date', axis = 1))

In [122]:
train_model(winter_variables, winter_target, model_lr)

0.6748279574884495

In [123]:
winter_scores = get_scores(winter_variables, winter_target)
winter_scores

0.7481704006239691

In [124]:
wind_temp_calculator(winter_variables, winter_target)

0.6118715987591691

In [125]:
temp_calculator(winter_variables, winter_target)

0.6119662463552029

In [126]:
no_temp_calculator(winter_variables, winter_target)

0.38959466691065936

### NE Spring Calculations

In [127]:
df_spring = season_extractor(dfne, 3, 20, 6, 21)
df_spring.shape

(470, 8)

In [ ]:
sns.pairplot(df_spring)

In [128]:
spring_variables = df_spring.drop(['Local date','Demand Forecast','Demand'], axis = 'columns')
spring_target = df_spring['Demand']
print(spring_target.head())
spring_variables.shape

262    305109
263    336706
264    327791
265    319884
266    324912
Name: Demand, dtype: int64


(470, 5)

In [129]:
train_model(spring_variables, spring_target, model_lr)

0.09058561460404357

In [130]:
spring_scores = get_scores(spring_variables, spring_target)
spring_scores

0.04706603657019803

In [131]:
wind_temp_calculator(spring_variables, spring_target)

-0.06104974574779143

In [132]:
temp_calculator(spring_variables, spring_target)

-0.09185561245062468

In [133]:
no_temp_calculator(spring_variables, spring_target)

0.04390241427829966

### NE Fall Calculations

In [134]:
df_fall = season_extractor(dfne, 9, 22, 12, 21)
df_fall.shape

(546, 8)

In [ ]:
sns.pairplot(df_fall)

In [ ]:
fall_variables = df_fall.drop(['Local date','Demand Forecast','Demand'], axis = 'columns')
fall_target = df_fall['Demand']
print(fall_target.head())
fall_variables.head()

In [136]:
train_model(fall_variables, fall_target, model_lr)

0.31278226208035564

In [137]:
fall_scores = get_scores(fall_variables, fall_target)
fall_scores

0.500219463001923

In [138]:
wind_temp_calculator(fall_variables, fall_target)

0.4083948862049005

In [139]:
temp_calculator(fall_variables, fall_target)

0.41064244040714637

In [140]:
no_temp_calculator(fall_variables, fall_target)

0.17547069630208978

## South East Electrical Region

In [141]:
dfs = pd.read_excel('NESE Demand.xlsx', 'SE_Daily', parse_dates = ['Local date'])
print(dfs.shape)
dfs.head()

(2009, 10)


,Region,Local date,Time zone,DF,D,GHI,Wind Speed,Relative Humidity,Temperature,Pressure
0,SE,2015-07-01,Central,765886.0,723871,272.243056,1.179167,85.007604,25.416667,998.020833
1,SE,2015-07-02,Central,748062.0,730320,219.875000,1.163194,89.075903,24.861111,996.805556
2,SE,2015-07-03,Central,716503.0,700726,235.541667,1.104514,87.619514,25.034722,996.562500
3,SE,2015-07-04,Central,691767.0,630155,232.593750,0.942361,92.207431,24.329861,998.472222
4,SE,2015-07-05,Central,671266.0,613549,188.934028,0.695833,90.758611,23.732639,999.791667


In [142]:
dfsee = dfs.drop(columns = ['Region','Time zone'], axis = 1)
#sns.pairplot(dfsee)

In [143]:
#dfse = data_cleaner(dfsee)
dfse = dfsee
dfse.shape

(2009, 8)

In [48]:
df0 = dfse[dfse['D']<50000]
df0.describe()

,DF,D,GHI,Wind Speed,Relative Humidity,Temperature,Pressure
count,48.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,24638.145833,12751.693878,212.105300,0.926481,77.130249,21.705577,997.850907
std,5268.892394,2499.488691,66.021353,0.285820,9.331572,6.060427,3.911272
min,17829.000000,9465.000000,46.763889,0.406944,51.847535,5.121528,991.423611
25%,19260.500000,10238.000000,163.586806,0.718056,70.866319,17.034722,995.069444
50%,25385.500000,13116.000000,213.312500,0.912153,77.697917,22.399306,996.909722
75%,29821.750000,15224.000000,264.736111,1.065972,84.015312,26.906250,1000.312500
max,31964.000000,16435.000000,301.527778,1.607292,96.711076,29.121528,1005.902778


In [146]:
new_dfse=dfse[~(dfse['D']<50000)]
new_dfse[new_dfse['D']<200000]

,Local date,DF,D,GHI,Wind Speed,Relative Humidity,Temperature,Pressure


In [147]:
new_dfse.isnull().sum()

Local date           0
DF                   2
D                    0
GHI                  0
Wind Speed           0
Relative Humidity    0
Temperature          0
Pressure             0
dtype: int64

In [ ]:
sns.pairplot(new_dfse.drop(columns = 'DF'))

### SE - Summer Calculations

In [148]:
dfse_summer = season_extractor(new_dfse, 6, 21, 9, 22)
print(dfse_summer.shape)
se_summer_variables = dfse_summer.drop(['Local date','DF','D'], axis = 'columns')
se_summer_target = dfse_summer['D']
se_summer_variables.head()

(529, 8)


,GHI,Wind Speed,Relative Humidity,Temperature,Pressure
0,272.243056,1.179167,85.007604,25.416667,998.020833
1,219.875000,1.163194,89.075903,24.861111,996.805556
2,235.541667,1.104514,87.619514,25.034722,996.562500
3,232.593750,0.942361,92.207431,24.329861,998.472222
4,188.934028,0.695833,90.758611,23.732639,999.791667


In [149]:
train_model(se_summer_variables,se_summer_target,model_lr)

0.7952723767707475

In [150]:
se_summer_scores = get_scores(se_summer_variables, se_summer_target)
se_summer_scores

0.8212970382125557

In [151]:
wind_temp_calculator(se_summer_variables, se_summer_target)

0.8203661138644565

In [152]:
temp_calculator(se_summer_variables, se_summer_target)

0.8207336482615935

In [153]:
no_temp_calculator(se_summer_variables, se_summer_target)

0.35449173975823534

### SE Winter Calculations 

In [154]:
dfse_winter = new_dfse[new_dfse['Local date'].apply(lambda x: x.month==1 or x.month==2 or x.month==3 or x.month==12)]
dfse_winter = dfse_winter[dfse_winter['Local date'].apply(lambda x: x.day<=21 if x.month==3 else True)]
dfse_winter = dfse_winter[dfse_winter['Local date'].apply(lambda x: x.day>=21 if x.month==12 else True)]
print(dfse_winter.shape)
se_winter_variables = dfse_winter.drop(['Local date','DF','D'], axis = 1)
se_winter_target = dfse_winter['D']
print(se_winter_variables.shape, se_winter_target.shape)

(457, 8)
(457, 5) (457,)


In [155]:
train_model(se_winter_variables,se_winter_target,model_lr)

0.7591416541366603

In [156]:
se_winter_scores = get_scores(se_winter_variables, se_winter_target)
se_winter_scores

0.74182660929265

In [157]:
wind_temp_calculator(se_winter_variables, se_winter_target)

0.7458233202226108

In [158]:
temp_calculator(se_winter_variables, se_winter_target)

0.7453072431271939

In [159]:
no_temp_calculator(se_winter_variables, se_winter_target)

0.6727907744791056

### SE Spring Calculations

In [160]:
dfse_spring = season_extractor(new_dfse, 3, 20, 6, 21)
dfse_spring.shape

(470, 8)

In [161]:
se_spring_variables = dfse_spring.drop(['Local date','DF','D'], axis = 1)
se_spring_target = dfse_spring['D']
print(se_spring_variables.shape, se_spring_target.shape)

(470, 5) (470,)


In [162]:
train_model(se_spring_variables,se_spring_target,model_lr)

0.6873201559661711

In [163]:
se_spring_scores = get_scores(se_spring_variables, se_spring_target)
se_spring_scores

0.754646124220218

In [164]:
wind_temp_calculator(se_spring_variables, se_spring_target)

0.7326784424954031

In [165]:
temp_calculator(se_spring_variables, se_spring_target)

0.7336818769498674

In [166]:
no_temp_calculator(se_spring_variables, se_spring_target)

0.05632619881897627

### SE Fall Calculations

In [167]:
dfse_fall = season_extractor(new_dfse, 9, 22, 12, 21)
dfse_fall.shape

(531, 8)

In [168]:
se_fall_variables = dfse_fall.drop(['Local date','DF','D'], axis = 1)
se_fall_target = dfse_fall['D']
print(se_fall_variables.shape, se_fall_target.shape)

(531, 5) (531,)


In [169]:
train_model(se_fall_variables,se_fall_target,model_lr)

0.013239218121379293

In [170]:
se_fall_scores = get_scores(se_fall_variables, se_fall_target)
se_fall_scores

-0.0254180545809799

In [171]:
wind_temp_calculator(se_fall_variables, se_fall_target)

0.008034926120391472

In [172]:
temp_calculator(se_fall_variables, se_fall_target)

-0.005363319773057373

In [173]:
no_temp_calculator(se_fall_variables, se_fall_target)

-0.007333135969577187

### Saving Model 

In [244]:
import pickle
with open ('NESE Model.pickle','wb') as f:
    pickle.dump(ridge_reg,f)